# Neural Networks


This notebook trains a neural network to model the total shrimp catch. Since the datasize is relatively small, we do not expect these models to outperform our gradient boosting models. However, these are useful for the purposes of comparative analysis.






In [25]:
# Import necessary libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.display import clear_output
from sklearn.model_selection import GridSearchCV


In [26]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# Load the dataset into a Pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/SHRIMP_constant_gear.csv')

In [31]:
# Preprocess the data
def preprocess_data(df):


    # Drop irrelevant columns
    df = df.drop(['MILITARY_TIME_STR', 'TEMP_SSURF','HAULVALUE', 'TIME_MIL'],  axis=1)  # Add this line
    return df

# Preprocess and display the dataframe
df = preprocess_data(df)

In [32]:
def split_data(df):
  X= df[['DEPTH_EMAX',
              'TEMPSURF',
              'TEMPMID',
              'SALSURF',
              'SALMAX',
              'OXYSURF',
              'OXYMID',
              'OXYMAX',
              'CHLORSURF',
              'CHLORMID',
              'CHLORMAX',
              'TEMP_BOT',
              'DECSLAT',
              'DECSLON',
              'YEAR',
              'DAYOFYEAR',
              'MINUTES_SINCE_MIDNIGHT']]
  y = df['TOTAL_SHRIMP_WEIGHT']

  # Split into training and test sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=216)
  return X_train, X_test, y_train, y_test

# Split the data
X_train, X_test, y_train, y_test = split_data(df)

# Create a copy of X_train and insert y_train as a new column
X_tt = X_train.copy()
X_tt["TOTAL_SHRIMP_WEIGHT"] = y_train

# Now split the combined data again
X_tt, X_val, y_tt, y_val = split_data(X_tt)

In [33]:
# prompt: Find the mean squared error of the baseline model which just returns the mean of the training set

# Calculate the mean of the training target values
y_tt_mean = y_tt.mean()

# Create an array of predicted values, all equal to the mean
y_pred_baseline = np.full_like(y_val, y_tt_mean)

# Calculate the mean squared error for the baseline model
mse_baseline = mean_squared_error(y_val, y_pred_baseline)
print(f'Mean Squared Error of the baseline model: {mse_baseline}')

Mean Squared Error of the baseline model: 2.4884055228466515


# Neural Network

Here is an attempt to model the data with a neural network. Note that this does not use a validation set, so we will have to edit this if we wanted to use this model. However, since there are fewer than 10,000 rows of shrimp data, I don't think that a neural net will be best so this is really just an experiment to see how it does.

In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer


In [35]:
# prompt: Create a pipeline which imputes any missing features using a kNN imputer and then applies a StandardScaler to standardize the features

# Create a pipeline with KNNImputer and StandardScaler
pipeline = Pipeline([
    ('imputer', KNNImputer()),
    ('scaler', StandardScaler())
])

# Fit and transform the training data
X_tt_transformed = pipeline.fit_transform(X_tt)

# Transform the test data
X_val_transformed = pipeline.transform(X_val)


In [36]:
# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_tt_transformed.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Output layer with one neuron for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_tt_transformed, y_tt, epochs=60, batch_size=32, validation_split=0.2, verbose=1)

# Predict on the test set
y_pred = model.predict(X_val_transformed)

# Calculate Mean Squared Error
mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error on validation set: {mse}')


Epoch 1/60


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


138/138 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 2.9043 - val_loss: 2.2467
Epoch 2/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1.9784 - val_loss: 2.1061
Epoch 3/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.9521 - val_loss: 2.0190
Epoch 4/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.8845 - val_loss: 2.0023
Epoch 5/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.8956 - val_loss: 1.9541
Epoch 6/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.8486 - val_loss: 1.9421
Epoch 7/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.7845 - val_loss: 1.9027
Epoch 8/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.7700 - val_loss: 1.8869
Epoch 9/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.6712 - val_loss: 1.8732
Epoch 10/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.6878 - val_loss: 1.8761
Epoch 11/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.8273 - val_loss: 1.8757
Epoch 12/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.

In [41]:
# Define a second neural network model using hyperbolic tangent as an activity
model_tanh = Sequential()
model_tanh.add(Dense(64, input_dim=X_tt_transformed.shape[1], activation='tanh'))
model_tanh.add(Dense(32, activation='tanh'))
model_tanh.add(Dense(1))  # Output layer with one neuron for regression

# Compile the model
model_tanh.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history_tanh = model_tanh.fit(X_tt_transformed, y_tt, epochs=60, batch_size=32, validation_split=0.2, verbose=1)

# Predict on the test set
y_pred_tanh = model_tanh.predict(X_val_transformed)

# Calculate Mean Squared Error
mse = mean_squared_error(y_val, y_pred_tanh)
print(f'Mean Squared Error on validation set: {mse}')

Epoch 1/60


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


138/138 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 3.1008 - val_loss: 2.1425
Epoch 2/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.2467 - val_loss: 2.1235
Epoch 3/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.0299 - val_loss: 2.0643
Epoch 4/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.0541 - val_loss: 2.0386
Epoch 5/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.9824 - val_loss: 2.0232
Epoch 6/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.9302 - val_loss: 2.0203
Epoch 7/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.9541 - val_loss: 1.9691
Epoch 8/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.8606 - val_loss: 1.9874
Epoch 9/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.0029 - val_loss: 1.9989
Epoch 10/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.9168 - val_loss: 1.9456
Epoch 11/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.8316 - val_loss: 1.9565
Epoch 12/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1

In [22]:
# prompt: Change the Neural network in the previous model to a convolutional neural network

from tensorflow.keras.layers import Conv1D, Flatten, MaxPooling1D

# Reshape the input data for CNN (samples, features, 1)
X_tt_transformed_cnn = X_tt_transformed.reshape(X_tt_transformed.shape[0], X_tt_transformed.shape[1], 1)
X_val_transformed_cnn = X_val_transformed.reshape(X_val_transformed.shape[0], X_val_transformed.shape[1], 1)

# Define the CNN model
model_cnn = Sequential()
model_cnn.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_tt_transformed_cnn.shape[1], 1)))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Conv1D(64, kernel_size=3, activation='relu'))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(32, activation='relu'))
model_cnn.add(Dense(1))  # Output layer

# Compile the model
model_cnn.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history_cnn = model_cnn.fit(X_tt_transformed_cnn, y_tt, epochs=60, batch_size=32, validation_split=0.2, verbose=1)

# Predict on the test set
y_pred_cnn = model_cnn.predict(X_val_transformed_cnn)

# Calculate Mean Squared Error
mse_cnn = mean_squared_error(y_val, y_pred_cnn)
print(f'Mean Squared Error on validation set (CNN): {mse_cnn}')


Epoch 1/60


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


138/138 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 2.9521 - val_loss: 2.5709
Epoch 2/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2.2929 - val_loss: 2.3843
Epoch 3/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.2754 - val_loss: 2.3543
Epoch 4/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 2.0953 - val_loss: 2.2990
Epoch 5/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.1727 - val_loss: 2.3015
Epoch 6/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.1104 - val_loss: 2.3401
Epoch 7/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.3075 - val_loss: 2.3188
Epoch 8/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.1966 - val_loss: 2.2620
Epoch 9/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.0046 - val_loss: 2.2672
Epoch 10/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.8827 - val_loss: 2.3131
Epoch 11/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.9291 - val_loss: 2.2719
Epoch 12/60
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2